In [ ]:
import numpy as np
from matplotlib import pyplot as plt
from scipy.signal import convolve

%matplotlib inline
%config InlineBackend.figure_format='retina'

Define the function to compute the kernel given the weights and the degree of the polynomial


In [ ]:
def compute_LPA_kernel(w, N):
    """
    Compute the LPA (Local Polynomial Approximation) kernel for a given weight vector and polynomial degree.

    Parameters:
    - w: numpy array of weights (length M), centered at position 0
    - N: degree of the polynomial approximation

    Returns:
    - g: numpy array representing the LPA kernel (same length as w)
    """
    M = len(w)

    # generate the inverse of weights
    winv = np.zeros_like(w)
    winv[w != 0] = 1 / w[w != 0]

    # set to zero weights that are inf
    winv[np.isinf(winv)] = 0

    # define the weight matrices
    W = np.diag(w)
    Winv = np.diag(winv)

    # Define the matrix T containing polynomials sampled over the window
    t = np.arange(M) / (M - 1)  # normalized time over the window [0, 1]
    T = np.zeros((M, N + 1))
    for i in range(N + 1):
        T[:, i] = t**i

    # compute the qr decomposition of WT
    Q, R = np.linalg.qr(W @ T)

    # define Qtilde
    Qtilde = Winv @ Q

    # adjust Qtilde with the weights matrix squared
    W2Qtilde = W @ W @ Qtilde

    # select the central row of W2Qtilde
    row = int((M - 1) / 2)

    # compute the kernel (first column of Q corresponds to the constant term)
    g = W2Qtilde[row, 0] * Qtilde[:, 0]

    # flipping, since it is used in convolution
    g = np.flip(g)

    return g

In [ ]:
def lpa_kernel(support, degree, weights=None):
    """
    Compute LPA convolution kernel

    Parameters:
    support: array of spatial support locations
    degree: polynomial degree
    weights: optional weight vector

    Returns:
    h: convolution kernel
    """

    M = len(support)
    L = degree

    # Design matrix
    T = np.zeros((M, L + 1))
    for i in range(M):
        for j in range(L + 1):
            T[i, j] = support[i] ** j

    # Apply weights if provided
    if weights is not None:
        W = np.diag(weights)
        T = W @ T

    # QR decomposition
    Q, R = qr(T, mode="economic")

    # Central index
    ic = M // 2

    # Compute kernel
    e_ic = np.zeros(M)
    e_ic[ic] = 1
    h = Q @ Q.T @ e_ic

    return h

## LPA-ICI

Set the LPA-ICI parameters


In [ ]:
# maximum degree of polynomial used for fitting
N = 5

# parameter for the confidence intervals in the ICI rule
Gamma = 2

# Set all the scale values
hmax = 51
all_h = np.arange(1, hmax + 1)

Generate the signal


In [ ]:
LENGTH = 1000

ty = np.linspace(0, 1, LENGTH)
y = np.sin(2 / (ty + 0.05))

#  noise standard deviation
sigma = 0.2

# noisy signal
s = y + sigma * np.random.normal(size=LENGTH)


plt.figure()
plt.plot(ty, s, "r.")
plt.plot(ty, y, "k--", linewidth=2)
plt.grid()
plt.legend(["noisy", "original"])
plt.title("Input Signal")

Generate the LPA kernels for all the scale. Use centered weights.


In [ ]:
all_g = []
for i in range(len(all_h)):
    h = all_h[i]
    # define the weights for the scale h (symmetric, centered)
    w = np.zeros(2 * hmax + 1)
    w[hmax - h : hmax + h + 1] = 1

    # compute and store the kernel g
    g = compute_LPA_kernel(w, N)

    all_g.append(g)

Initialize all the variables for the ICI rule


In [ ]:
# initialize the estimate
yhat = np.zeros_like(s)

# initialize the vector containing the best scale for each sample
best_scale = np.ones(LENGTH, dtype=int)

# initialize the lower and upper bound vectors
lower_bounds = -np.inf * np.ones(LENGTH)
upper_bounds = np.inf * np.ones(LENGTH)

Loop over all the scales


In [ ]:
for i, h in enumerate(all_h):
    g = all_g[i]

    # compute the estimate for the scale h
    yhat_h = convolve(s, g, mode="same")

    # compute the variance of the estimate
    var_h = sigma**2 * convolve(np.ones_like(s), g**2, mode="same")

    # compute the lower and upper bound of the confidence interval for the scale h
    lb = yhat_h - Gamma * np.sqrt(var_h)
    ub = yhat_h + Gamma * np.sqrt(var_h)

    # update the lower and upper bounds (intersection)
    lower_bounds = np.maximum(lower_bounds, lb)
    upper_bounds = np.minimum(upper_bounds, ub)

    # identify for which samples h is the best scale according to the
    # ICI rule and update the best_scale vector accordingly
    valid_ici = lower_bounds <= upper_bounds
    best_scale[valid_ici] = h

    # update the estimate
    yhat[valid_ici] = yhat_h[valid_ici]

Use the best scale for each sample to compute the final estimates


In [ ]:
yhat_final = yhat.copy()

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12, 7))
ax[0].plot(ty, s, "r.")
ax[0].plot(ty, y, "k--", linewidth=3)
ax[0].plot(ty, yhat_final, "m-", linewidth=3, color="blue")
ax[0].grid()
ax[0].legend(["noisy", "original", "LPA-ICI estimate"])
ax[0].set_title(f"N = {N:d}")

ax[1].plot(ty, best_scale, "r.")
ax[1].set_title("Scale selected by ICI rule")
ax[1].grid()

fig.tight_layout()
plt.show()

## LPA-ICI with Aggregation

Set the parameters


In [ ]:
# maximum degree of polynomial used for fitting
N = 1

# parameter for the confidence intervals in the ICI rule
Gamma = 2

# Set all the scale values
hmax = 51
all_h = np.arange(1, hmax + 1)

Generate synthetic signal signal


In [ ]:
LENGTH = 1000
ty = np.linspace(0, 1, LENGTH)
y = 8 * ty**2 - 2 * ty + 2
y[ty > 0.5] = y[ty > 0.5] + 7

#  noise standard deviation
sigma = 0.3

# noisy signal
s = y + sigma * np.random.normal(size=LENGTH)


plt.figure()
plt.plot(ty, s, "r.")
plt.plot(ty, y, "k--", linewidth=2)
plt.grid()
plt.legend(["noisy", "original"])
plt.title("Input Signal")

Generate the LPA kernels for all the scale for both left and right windows


In [ ]:
all_g_left = []
all_g_right = []

for i, h in enumerate(all_h):
    # define the weights for the scale h (left)
    w = np.zeros(2 * hmax + 1)
    w[hmax - h : hmax + 1] = 1  # left window
    g_left = compute_LPA_kernel(w, N)
    all_g_left.append(g_left)

    # define the weights for the scale h (right)
    w = np.zeros(2 * hmax + 1)
    w[hmax : hmax + h + 1] = 1  # right window
    g_right = compute_LPA_kernel(w, N)
    all_g_right.append(g_right)

Use the LPA-ICI to compute the estimate based on the **left** kernels


In [ ]:
# initialize the left estimate
yhat_left = np.zeros_like(s)

# initialize the lower and upper bound vectors
lower_bounds = -np.inf * np.ones(LENGTH)
upper_bounds = np.inf * np.ones(LENGTH)

# intialize the vector containing the variance of the estimator for each sample
var_left = np.zeros_like(s)

for i, h in enumerate(all_h):
    g = all_g_left[i]

    # compute the estimate for the scale h
    yhat_h = convolve(s, g, mode="same")

    # compute the variance of the estimate
    var_h = sigma**2 * convolve(np.ones_like(s), g**2, mode="same")

    # compute the lower and upper bound of the confidence interval for the scale h
    lb = yhat_h - Gamma * np.sqrt(var_h)
    ub = yhat_h + Gamma * np.sqrt(var_h)

    # update the lower and upper bounds
    lower_bounds = np.maximum(lower_bounds, lb)
    upper_bounds = np.minimum(upper_bounds, ub)

    # identify valid ICI points
    valid_ici = lower_bounds <= upper_bounds

    # update the estimate
    yhat_left[valid_ici] = yhat_h[valid_ici]

    # update the variance
    var_left[valid_ici] = var_h[valid_ici]

Use the LPA-ICI to compute the estimate based on the **right** kernels


In [ ]:
yhat_right = np.zeros_like(s)
# initialize the lower and upper bound vectors
lower_bounds = -np.inf * np.ones(LENGTH)
upper_bounds = np.inf * np.ones(LENGTH)

# intialize the vector containing the variance of the estimator for each sample
var_right = np.zeros_like(s)

for i, h in enumerate(all_h):
    g = all_g_right[i]  # Fixed: was using all_g_left[i]

    # compute the estimate for the scale h
    yhat_h = convolve(s, g, mode="same")

    # compute the variance of the estimate
    var_h = sigma**2 * convolve(np.ones_like(s), g**2, mode="same")

    # compute the lower and upper bound of the confidence interval for the scale h
    lb = yhat_h - Gamma * np.sqrt(var_h)
    ub = yhat_h + Gamma * np.sqrt(var_h)

    # update the lower and upper bounds
    lower_bounds = np.maximum(lower_bounds, lb)
    upper_bounds = np.minimum(upper_bounds, ub)

    # identify valid ICI points
    valid_ici = lower_bounds <= upper_bounds

    # update the estimate
    yhat_right[valid_ici] = yhat_h[valid_ici]

    # update the variance
    var_right[valid_ici] = var_h[valid_ici]

Perform the aggregation


In [ ]:
weight_left = 1 / (var_left + 1e-10)  # add small epsilon to avoid division by zero
weight_right = 1 / (var_right + 1e-10)
total_weight = weight_left + weight_right

yhat_aggr = (weight_left * yhat_left + weight_right * yhat_right) / total_weight

In [ ]:
plt.figure(figsize=(10, 10))
plt.plot(ty, s, "r.")
plt.plot(ty, y, "k--", linewidth=3)
plt.plot(ty, yhat_right, "m-", linewidth=3)
plt.plot(ty, yhat_left, "g-", linewidth=3)
plt.plot(ty, yhat_aggr, "b-", linewidth=3)
plt.grid()
plt.legend(
    ["noisy", "original", "right estimate", "left estimate", "aggregated estimate"]
)
plt.title(f"N = {N:d}")
